# AutoChip Tutorial Assignment — example1.ipynb (ChipChat Example 1)

## Example 1: binary_to_bcd (Binary → BCD Converter)

This notebook is self-contained and includes:
- `config.json` (full content printed + saved)
- Initial module function description + module template used by AutoChip
- Testbench code (included verbatim from ChipChat source; printed + saved locally)
- Full AutoChip generation trajectory for final correct RTL (saved to `trajectory.log`)
- Exact `iverilog/vvp` command and the passing output

All generated Verilog, logs, and copied configuration/testbench content are saved inside `/content/example1/`.

## Install dependencies


In [17]:
!pip -q install anthropic
!apt-get -qq update
!apt-get -qq install -y iverilog

import os, textwrap, json, subprocess, re, sys, time
print(" Dependencies installed.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
 Dependencies installed.


##Load  API key from Secrets

In [18]:
from google.colab import userdata
import os

ANTHROPIC_API_KEY = userdata.get("LLM4ChipDesign")
if not ANTHROPIC_API_KEY:
    raise RuntimeError("Missing secret 'LLM4ChipDesign' in Colab Secrets (or not enabled for this notebook).")

os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
print(" Anthropic API key loaded from Colab Secrets.")

 Anthropic API key loaded from Colab Secrets.


In [20]:
ROOT = "/content/example1"
os.makedirs(ROOT, exist_ok=True)
os.chdir(ROOT)

print(" Working directory:", os.getcwd())
!ls -la

 Working directory: /content/example1
total 12
drwxr-xr-x 3 root root 4096 Feb 13 23:30 .
drwxr-xr-x 1 root root 4096 Feb 13 23:07 ..
drwxr-xr-x 2 root root 4096 Feb 13 23:30 .ipynb_checkpoints


## Fetch the ChipChat testbench and PRINT it

In [21]:
TB_URL = "https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v"

!curl -L -o binary_to_bcd_tb.v "$TB_URL"

print(" Downloaded testbench from ChipChat source URL:")
print(TB_URL)

print("\n binary_to_bcd_tb.v ")
with open("binary_to_bcd_tb.v", "r") as f:
    tb_text = f.read()
print(tb_text)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1078  100  1078    0     0   6198      0 --:--:-- --:--:-- --:--:--  6231
 Downloaded testbench from ChipChat source URL:
https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v

 binary_to_bcd_tb.v 
`timescale 1ns / 1ps

module tb_binary_to_bcd_converter;

reg [4:0] binary_input;
wire [7:0] bcd_output;

binary_to_bcd_converter uut (
    .binary_input(binary_input),
    .bcd_output(bcd_output)
);

integer i;
reg [4:0] test_binary;
reg [7:0] expected_bcd;

initial begin
    $display("Testing Binary-to-BCD Converter...");

    for (i = 0; i < 32; i++) begin
        test_binary = i;
        binary_input = test_binary;

        // Calculate expected BCD output
        expected_bcd[3:0] = test_binary % 10;
        expected_bcd[7:4] = test_

## Testbench notes (commented explanation) + invocation

- The testbench file is `binary_to_bcd_tb.v` (downloaded from the ChipChat notebook’s source URL).
- It instantiates the DUT and checks whether the BCD output matches the expected tens/ones encoding for all relevant inputs.
- We invoke it exactly using:

```bash
iverilog -g2012 -o sim.out out/binary_to_bcd.sv binary_to_bcd_tb.v
vvp sim.out


## Write config.json

In [22]:
config = {
  "general": {
    "name": "binary_to_bcd",                 # generated RTL saved as out/binary_to_bcd.sv
    "prompt": "prompt.txt",
    "testbench": "binary_to_bcd_tb.v",
    "iterations": 8,
    "num_candidates": 3,
    "outdir": "out",
    "log": "trajectory.log",
    "model_family": "Anthropic",
    "model_id": "claude-3-5-haiku-latest"
  }
}

with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

print("config.json")
print(json.dumps(config, indent=2))

config.json
{
  "general": {
    "name": "binary_to_bcd",
    "prompt": "prompt.txt",
    "testbench": "binary_to_bcd_tb.v",
    "iterations": 8,
    "num_candidates": 3,
    "outdir": "out",
    "log": "trajectory.log",
    "model_family": "Anthropic",
    "model_id": "claude-3-5-haiku-latest"
  }
}


## Initial module function description + module template (used by AutoChip)

### Function description
Implement a synthesizable Verilog module that converts:
- **Input:** 5-bit binary value (`binary_input`), representing 0–31
- **Output:** 8-bit BCD (`bcd_output`)
  - `bcd_output[7:4]` = tens digit (0–3)
  - `bcd_output[3:0]` = ones digit (0–9)

### Interface (must match exactly)
```verilog
module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);


## Save + print the initial module template

In [23]:
module_template = textwrap.dedent(r"""
`timescale 1ns/1ps
// binary_to_bcd_converter: 5-bit binary (0..31) -> 8-bit BCD (tens in [7:4], ones in [3:0])
module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);
    // TODO: implement conversion (synthesizable)
endmodule
""").strip() + "\n"

with open("binary_to_bcd_template.sv", "w") as f:
    f.write(module_template)

print("Initial module template: binary_to_bcd_template.sv ")
print(module_template)

Initial module template: binary_to_bcd_template.sv 
`timescale 1ns/1ps
// binary_to_bcd_converter: 5-bit binary (0..31) -> 8-bit BCD (tens in [7:4], ones in [3:0])
module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);
    // TODO: implement conversion (synthesizable)
endmodule



## Save + print the AutoChip prompt

In [24]:
prompt = textwrap.dedent(r"""
You are an autocomplete engine for synthesizable Verilog.

Goal:
Write a synthesizable Verilog module that converts a 5-bit binary input (0..31)
into an 8-bit BCD output:
- bcd_output[7:4] = tens digit (0..3)
- bcd_output[3:0] = ones digit (0..9)

The module interface MUST match exactly:

module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);

Rules:
- Synthesizable RTL only (no delays #, no initial blocks, no $display).
- Combinational logic is acceptable (always @* or continuous assigns).
- Ensure correct BCD encoding for all inputs 0..31.
- Return ONLY the Verilog module in a single ```verilog``` code block.
- Do NOT output a testbench.

Use/complete this template:

```verilog
`timescale 1ns/1ps
module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);
    // TODO: implement conversion (synthesizable)
endmodule
""").strip() + "\n"

with open("prompt.txt", "w") as f:
    f.write(prompt)

print("prompt.txt")
print(prompt)


prompt.txt
You are an autocomplete engine for synthesizable Verilog.

Goal:
Write a synthesizable Verilog module that converts a 5-bit binary input (0..31)
into an 8-bit BCD output:
- bcd_output[7:4] = tens digit (0..3)
- bcd_output[3:0] = ones digit (0..9)

The module interface MUST match exactly:

module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);

Rules:
- Synthesizable RTL only (no delays #, no initial blocks, no $display).
- Combinational logic is acceptable (always @* or continuous assigns).
- Ensure correct BCD encoding for all inputs 0..31.
- Return ONLY the Verilog module in a single ```verilog``` code block.
- Do NOT output a testbench.

Use/complete this template:

```verilog
`timescale 1ns/1ps
module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);
    // TODO: implement conversion (synthesizable)
endmodule



## AutoChip loop (Anthropic Claude) + compile/sim helpers

In [25]:
import anthropic
from abc import ABC, abstractmethod

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file
        if self.log_file:
            open(self.log_file, 'w').close()

    def add_message(self, role, content):
        self.messages.append({'role': role, 'content': content})
        if self.log_file:
            with open(self.log_file, 'a') as f:
                f.write(f"{role}: {content}\n")

    def get_messages(self):
        return self.messages

class AbstractLLM(ABC):
    @abstractmethod
    def generate(self, conversation: Conversation, num_candidates=1):
        pass

class ClaudeLLM(AbstractLLM):
    def __init__(self, model_id="claude-3-5-haiku-latest"):
        self.client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
        self.model_id = model_id

    def generate(self, conversation: Conversation, num_candidates=1):
        system_prompt = ""
        msg_list = []
        for m in conversation.get_messages():
            if m["role"] == "system":
                system_prompt = m["content"]
            elif m["role"] in ("user", "assistant"):
                msg_list.append({"role": m["role"], "content": m["content"]})

        outputs = []
        for _ in range(num_candidates):
            resp = self.client.messages.create(
                model=self.model_id,
                max_tokens=2200,
                temperature=0.2,
                system=system_prompt,
                messages=msg_list
            )
            outputs.append(resp.content[0].text)
        return outputs

def extract_first_verilog_codeblock(text):
    m = re.search(r"```verilog(.*?)```", text, re.S)
    if m:
        return m.group(1).strip() + "\n"
    m2 = re.search(r"```(.*?)```", text, re.S)
    if m2:
        return m2.group(1).strip() + "\n"
    return text.strip() + "\n"

def write_text(path, s):
    with open(path, "w") as f:
        f.write(s)

def compile_and_sim(outdir, sv_file, tb_file):
    os.makedirs(outdir, exist_ok=True)
    exe_file = os.path.join(outdir, "sim.out")

    compile_cmd = f"iverilog -g2012 -o {exe_file} {sv_file} {tb_file}"
    sim_cmd = f"vvp {exe_file}"

    comp = subprocess.run(compile_cmd, shell=True, capture_output=True, text=True)
    if comp.returncode != 0:
        return False, compile_cmd, sim_cmd, comp.stdout, comp.stderr, "", ""

    sim = subprocess.run(sim_cmd, shell=True, capture_output=True, text=True)
    ok = (sim.returncode == 0) and (("PASS" in sim.stdout) or ("pass" in sim.stdout))
    return ok, compile_cmd, sim_cmd, comp.stdout, comp.stderr, sim.stdout, sim.stderr

def verilog_loop(design_prompt, out_sv_path, tb_file, max_iterations,
                 model_id="claude-3-5-haiku-latest", num_candidates=3, log="trajectory.log"):

    conv = Conversation(log_file=log)
    conv.add_message("system",
        "You generate synthesizable Verilog. Return ONLY the requested module in a single ```verilog``` code block."
    )
    conv.add_message("user", design_prompt)

    model = ClaudeLLM(model_id=model_id)

    for it in range(1, max_iterations + 1):
        print(f"\n Iteration {it}/{max_iterations}")
        candidates = model.generate(conv, num_candidates=num_candidates)

        for k, txt in enumerate(candidates, start=1):
            parsed = extract_first_verilog_codeblock(txt)
            write_text(out_sv_path, parsed)

            ok, compile_cmd, sim_cmd, c_out, c_err, s_out, s_err = compile_and_sim(
                outdir=os.path.dirname(out_sv_path),
                sv_file=out_sv_path,
                tb_file=tb_file
            )

            print(f"\n--- Candidate {k} ---")
            print("Compile cmd:", compile_cmd)
            if c_err.strip():
                print("Compile stderr:\n", c_err)
            if s_out.strip():
                print("Sim stdout:\n", s_out)

            if ok:
                print(f"✅ PASS on iteration {it}, candidate {k}")
                return parsed

            feedback = "Compilation or simulation failed.\n"
            if c_err.strip():
                feedback += "Compiler error:\n" + c_err + "\n"
            if s_out.strip():
                feedback += "Simulator output:\n" + s_out + "\n"
            if s_err.strip():
                feedback += "Simulator stderr:\n" + s_err + "\n"

            conv.add_message("assistant", txt)
            conv.add_message("user",
                "Fix the Verilog module to satisfy the testbench. "
                "Do NOT write a testbench. "
                "Return ONLY the corrected module in a single ```verilog``` code block.\n\n" + feedback
            )

    raise RuntimeError("AutoChip loop ended without a passing solution.")

print("✅ AutoChip utilities loaded (Anthropic / Haiku).")

✅ AutoChip utilities loaded (Anthropic / Haiku).


## Run AutoChip + save final RTL + show PASS run

In [26]:
with open("config.json", "r") as f:
    cfg = json.load(f)["general"]

module_base = cfg["name"]                  # binary_to_bcd
tb_file = cfg["testbench"]                 # binary_to_bcd_tb.v
iterations = int(cfg["iterations"])
num_candidates = int(cfg["num_candidates"])
outdir = cfg["outdir"]
logfile = cfg["log"]
model_id = cfg["model_id"]

with open(cfg["prompt"], "r") as f:
    design_prompt = f.read()

os.makedirs(outdir, exist_ok=True)
out_sv_path = os.path.join(outdir, f"{module_base}.sv")

start = time.time()
final_sv = verilog_loop(
    design_prompt=design_prompt,
    out_sv_path=out_sv_path,
    tb_file=tb_file,
    max_iterations=iterations,
    model_id=model_id,
    num_candidates=num_candidates,
    log=logfile
)
elapsed = time.time() - start
print(f"\nDone. Time elapsed: {elapsed:.2f}s")

# Save a copy in root folder too (extra self-contained evidence)
with open(f"{module_base}_FINAL.sv", "w") as f:
    f.write(final_sv)

print(f" Final RTL saved to: {out_sv_path}")
print(f" Final RTL copied to: {module_base}_FINAL.sv")


 Iteration 1/8


/tmp/ipython-input-3034649060.py:41: DeprecationWarning: The model 'claude-3-5-haiku-latest' is deprecated and will reach end-of-life on February 19th, 2026.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  resp = self.client.messages.create(



--- Candidate 1 ---
Compile cmd: iverilog -g2012 -o out/sim.out out/binary_to_bcd.sv binary_to_bcd_tb.v
Sim stdout:
 Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed!

✅ PASS on iteration 1, candidate 1

Done. Time elapsed: 27.51s
 Final RTL saved to: out/binary_to_bcd.sv
 Final RTL copied to: binary_to_bcd_FINAL.sv


## exact iverilog/vvp command + passing output

In [27]:
exe_file = os.path.join(outdir, "sim_final.out")
compile_cmd = f"iverilog -g2012 -o {exe_file} {out_sv_path} {tb_file}"
sim_cmd = f"vvp {exe_file}"

print("REQUIRED SIM COMMANDS ")
print("iverilog command:\n", compile_cmd)
print("\nvvp command:\n", sim_cmd)

comp = subprocess.run(compile_cmd, shell=True, capture_output=True, text=True)
print("\n compile stdout \n", comp.stdout)
print("\n compile stderr \n", comp.stderr)
assert comp.returncode == 0, "Compile failed — see stderr above."

sim = subprocess.run(sim_cmd, shell=True, capture_output=True, text=True)
print("\n sim stdout \n", sim.stdout)
print("\n sim stderr \n", sim.stderr)
assert sim.returncode == 0 and (("PASS" in sim.stdout) or ("pass" in sim.stdout)), "Simulation did not PASS."
print("\n✅ PASS evidence captured above.")


REQUIRED SIM COMMANDS 
iverilog command:
 iverilog -g2012 -o out/sim_final.out out/binary_to_bcd.sv binary_to_bcd_tb.v

vvp command:
 vvp out/sim_final.out

 compile stdout 
 

 compile stderr 
 

 sim stdout 
 Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed!


 sim stderr 
 

✅ PASS evidence captured above.


## show trajectory log (saved fully) + key notes

In [28]:
print(" REQUIRED: FULL TRAJECTORY LOG ")
with open(logfile, "r") as f:
    traj = f.read()

print(traj[:12000])
if len(traj) > 12000:
    print("\n... (log truncated in display; full log saved to trajectory.log) ...\n")
    print(traj[-4000:])

print("\nKey notes:")
print("- trajectory.log contains system prompt, user prompt, model outputs, and feedback across iterations.")
print(f"- Final passing RTL is: {out_sv_path}")
print(f"- Copy of final RTL is: {module_base}_FINAL.sv")


 REQUIRED: FULL TRAJECTORY LOG 
system: You generate synthesizable Verilog. Return ONLY the requested module in a single ```verilog``` code block.
user: You are an autocomplete engine for synthesizable Verilog.

Goal:
Write a synthesizable Verilog module that converts a 5-bit binary input (0..31)
into an 8-bit BCD output:
- bcd_output[7:4] = tens digit (0..3)
- bcd_output[3:0] = ones digit (0..9)

The module interface MUST match exactly:

module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);

Rules:
- Synthesizable RTL only (no delays #, no initial blocks, no $display).
- Combinational logic is acceptable (always @* or continuous assigns).
- Ensure correct BCD encoding for all inputs 0..31.
- Return ONLY the Verilog module in a single ```verilog``` code block.
- Do NOT output a testbench.

Use/complete this template:

```verilog
`timescale 1ns/1ps
module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0

## Show folder contents

In [29]:
print(" Folder contents:")
!find /content/example1 -maxdepth 2 -type f -print

 Folder contents:
/content/example1/config.json
/content/example1/my_design.vcd
/content/example1/trajectory.log
/content/example1/out/binary_to_bcd.sv
/content/example1/out/sim.out
/content/example1/out/sim_final.out
/content/example1/binary_to_bcd_template.sv
/content/example1/binary_to_bcd_tb.v
/content/example1/binary_to_bcd_FINAL.sv
/content/example1/prompt.txt


### Testbench code (commented)

In [30]:
with open("binary_to_bcd_tb.v","r") as f:
    orig = f.read()

header = """`timescale 1ns/1ps
// NOTE: This testbench is sourced from the ChipChat notebook example.
// It instantiates binary_to_bcd_converter and checks output BCD for test vectors.
// We compile and run it with:
//   iverilog -g2012 -o sim.out out/binary_to_bcd.sv binary_to_bcd_tb_commented.v
//   vvp sim.out
"""
with open("binary_to_bcd_tb_commented.v","w") as f:
    f.write(header + "\n" + orig)

print(orig[:500])


`timescale 1ns / 1ps

module tb_binary_to_bcd_converter;

reg [4:0] binary_input;
wire [7:0] bcd_output;

binary_to_bcd_converter uut (
    .binary_input(binary_input),
    .bcd_output(bcd_output)
);

integer i;
reg [4:0] test_binary;
reg [7:0] expected_bcd;

initial begin
    $display("Testing Binary-to-BCD Converter...");

    for (i = 0; i < 32; i++) begin
        test_binary = i;
        binary_input = test_binary;

        // Calculate expected BCD output
        expected_bcd[3:0] = test_bi
